## Imports

In [2]:
import os

import pandas as pd

from sqlalchemy import create_engine

from dotenv import load_dotenv
load_dotenv()

True

## Connection

In [4]:
username = os.getenv('POSTGRES_USERNAME')
password = os.getenv('POSTGRES_PASSWORD')


postgres_connection_string = "postgres://{username}:{password}@{host}:{port}/{database}?gssencmode=disable".format(
    username=username,
    password=password,
    host="localhost",
    port="5432",
    database="lahman_baseball"
)

engine = create_engine(postgres_connection_string)

## Table Retriever

In [25]:
def get_table(table_name):
    """This function uses the connection to lahman_baseball database to return a dataframe of all the data in the table"""
    sql = f'SELECT * FROM {table_name};'
    return pd.read_sql(sql, con=engine)

---
# Question 1: What range of years does the provided database cover?

### Database Documentation: http://www.seanlahman.com/files/database/readme2017.txt

### This documentation indicates that People, Batting, Pitching, and Fielding are the main tables

In [28]:
people_df = get_table('people')

batting_df = get_table('batting')

pitching_df = get_table('pitching')

fielding_df = get_table('fielding')

In [29]:
# Only birthyear in player table, not really 'baseball statistics data'

people_df.columns

Index(['playerid', 'birthyear', 'birthmonth', 'birthday', 'birthcountry',
       'birthstate', 'birthcity', 'deathyear', 'deathmonth', 'deathday',
       'deathcountry', 'deathstate', 'deathcity', 'namefirst', 'namelast',
       'namegiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalgame',
       'retroid', 'bbrefid'],
      dtype='object')

In [35]:
# Batting table:

print('Batting Table Year Min:', batting_df.yearid.min())
print()
print('Batting Table Year Max:', batting_df.yearid.max())

Batting Table Year Min: 1871

Batting Table Year Max: 2016


In [36]:
# Pitching table:

print('Pitching Table Year Min:', pitching_df.yearid.min())
print()
print('Pitching Table Year Max:', pitching_df.yearid.max())

Pitching Table Year Min: 1871

Pitching Table Year Max: 2016


In [37]:
# Fielding table:

print('Fielding Table Year Min:', fielding_df.yearid.min())
print()
print('Fielding Table Year Max:', fielding_df.yearid.max())

Fielding Table Year Min: 1871

Fielding Table Year Max: 2016


## Answer: The main tables cover 1871 - 2016
---

---
# Question 2: Find the name and height of the shortest player in the database. How many games did he play in? What is the name of the team for which he played?

In [38]:
people_df = get_table('people')

appearances_df = get_table('appearances')

teams_df = get_table('teams')

In [47]:
people_df[people_df.height == people_df.height.min()].loc[:,['playerid', 'namefirst', 'namelast', 'namegiven', 'height']]

,playerid,namefirst,namelast,namegiven,height
5843,gaedeed01,Eddie,Gaedel,Edward Carl,43.0


In [70]:
short_id = people_df[people_df.height == people_df.height.min()]['playerid'].to_string(index=False).strip()

In [76]:
print('Number of games played:', appearances_df[appearances_df.playerid == short_id]['g_all'].to_string(index=False))

Number of games played:  1


In [80]:
team_id = appearances_df[appearances_df.playerid == short_id]['teamid'].to_string(index=False).strip()

In [95]:
teams_df.name[teams_df.teamid == team_id].head(1).to_string(index=False).strip()

'St. Louis Browns'

## Answer: Eddie Gaedel was the shortest player. His height was 43.0 inches (see documentation for units). He played 1 game for the St. Louis Browns
---

---
# Question 3: Find all players in the database who played at Vanderbilt University. Create a list showing each player’s first and last names as well as the total salary they earned in the major leagues. Sort this list in descending order by the total salary earned. Which Vanderbilt player earned the most money in the majors?

In [98]:
people_df = get_table('people')

collegeplaying_df = get_table('collegeplaying')

schools_df = get_table('schools')

salaries_df = get_table('salaries')

In [104]:
# Getting the schoolid of Vanderbilt University from the Schools table

vandy_id = schools_df.schoolid[schools_df.schoolname == 'Vanderbilt University'].to_string(index=False).strip()

'vandy'

In [110]:
# Getting a list of playerids from collegeplaying table with the Vanderbilt Unviersity schoolid

vandy_players = collegeplaying_df[collegeplaying_df.schoolid == vandy_id].playerid.unique()

array(['alvarpe01', 'baxtemi01', 'chrisni01', 'colliwi01', 'corajo01',
       'embresl01', 'flahery01', 'grayso01', 'hendrha01', 'katama01',
       'lewisje01', 'madissc01', 'minormi01', 'mooresc01', 'mossma01',
       'pauljo01', 'priceda01', 'priorma01', 'richaan01', 'sandesc01',
       'sewelri01', 'sowerje01', 'willimi01', 'zeidjo01'], dtype=object)

In [113]:
# Making a dataframe by narrowing people down to only the columns of interest for players with playerid in vandy_players

vandy_people = people_df[people_df.playerid.isin(vandy_players)].loc[:,['playerid', 'namefirst', 'namelast']]

In [124]:
# Summing salaries by playerid across all years

salary_sum = salaries_df.groupby(by = 'playerid', as_index = False)['salary'].sum()

In [136]:
vandy_salaries = pd.merge(vandy_people, salary_sum, how = 'inner', on='playerid') \
                          .sort_values(by = ['salary']) \
                          .reset_index(drop = True)

vandy_salaries

,playerid,namefirst,namelast,salary
0,madissc01,Scotti,Madison,135000.0
1,sowerje01,Jeremy,Sowers,384800.0
2,chrisni01,Nick,Christiani,500000.0
3,katama01,Matt,Kata,1060000.0
4,lewisje01,Jensen,Lewis,1234000.0
5,grayso01,Sonny,Gray,1542500.0
6,baxtemi01,Mike,Baxter,2094418.0
7,pauljo01,Josh,Paul,2640000.0
8,flahery01,Ryan,Flaherty,4061000.0
9,corajo01,Joey,Cora,5622500.0


## Answer: David Price earned the most money in the majors. $81,851,296
---

In [137]:
engine.dispose()